# Grupo 3 del proyecto de data mining - Analisis de datos

***Authors : N.Matte, ...***

In [4]:
# Imports
import requests
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# I. Import data from the website

First, I download the HTML page where I can find the URLs of every monthly file I want to analyse.

In [2]:
# URL of the file I want to download
url_csv = "https://www.omie.es/en/file-access-list?parents%5B0%5D=/&parents%5B1%5D=Day-ahead%20Market&parents%5B2%5D=3.%20Curves&dir=Monthly%20files%20with%20aggregate%20supply%20and%20demand%20curves%20of%20Day-ahead%20market%20including%20bid%20units&realdir=curva_pbc_uof"

# Path where I want to save the file
path = "data/webpage_omie.html"

# GET request to try to access URL data
response = requests.get(url_csv)

# I make sure the request succeeded
if response.status_code == 200:
    # ... so I write the data in a local file, in the specified path
    with open(path, "wb") as f:
        f.write(response.content)
    print(f"File saved at {path}")
else:
    print(f"Request error code: {response.status_code}")


File saved at data/webpage_omie.html


Now, with the HTML, I need to extract the list containing the URLs of all curves files.

In [24]:
# I read the HTML file
with open(path, "r", encoding="utf-8") as file:
    html_content = file.read()

    # Motif avec des caractères génériques avant et après
    pattern = r".*curva_pbc_uof_*."

    # Recherche des occurrences qui correspondent au motif
    matches = re.findall(pattern, html_content)

    print("Correspondances trouvées:")
    if matches:
        print(len(matches), "occurences found on the webpage.")
    else:
        print("Aucune correspondance trouvée")

# I download the first file, just for a test


Correspondances trouvées:
76 occurences found on the webpage.


Now I need to process the matches to extract only the URLs I need.

In [ ]:
...

Finally, I can download every zip file.

In [28]:
# URL of the file I want to download
# url_csv = "https://www.omie.es/" + "/es/file-download?parents%5B0%5D=curva_pbc_uof&amp;filename=curva_pbc_uof_202402.zip"
url_csv = "https://www.omie.es/en/file-download?parents%5B0%5D=curva_pbc_uof&filename=curva_pbc_uof_202402.zip" 

# Nom du fichier à partir du lien
file_name = "first_file.zip"

# Récupérer le contenu du fichier ZIP
response = requests.get(url_csv)

if response.status_code == 200:
    # Sauvegarder le fichier localement
    with open(file_name, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):  # Télécharger par morceaux
            f.write(chunk)
    print(f"Téléchargement terminé : {file_name}")
else:
    print(f"Erreur lors du téléchargement : {response.status_code}")

Téléchargement terminé : first_file.zip
